In [1]:
from iMahir import Study

In [2]:
hebrew = '/Users/cody/Documents/vocabulary/hebrew.json'

In [3]:
heb = Study(hebrew)

preparing TF...
This is Text-Fabric 7.6.8
Api reference : https://annotation.github.io/text-fabric/Api/Fabric/

115 features found and 0 ignored
  0.00s loading features ...
   |     0.01s B gloss                from /Users/cody/text-fabric-data/etcbc/bhsa/tf/2017
   |     0.08s B freq_lex             from /Users/cody/text-fabric-data/etcbc/bhsa/tf/2017
  4.25s All features loaded/computed - for details use loadLog()
	connecting to online GitHub repo annotation/app-bhsa ... connected
Using TF-app in /Users/cody/text-fabric-data/annotation/app-bhsa/code:
	rv1.0=#d3cf8f0c2ab5d690a0fda14ea31c33da5c5c8483 (latest release)


hebrew ready for study.
this is session 8:
score 4: 4 terms
score 3: 184 terms
score 2: 10 terms
score 1: 2 terms
total: 200


In [4]:
heb.learn()

The following scores were changed 
2<-3 		 2
2->3 		 5

duration:  0:30:40.960979


<br>
<br>
<br>
<br>
<br>
<br>

### Notes

* solve discrepancy between יבשׁ and בושׁ

<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>